In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

In [ ]:
import pandas as pd
import cv2
import os.path
from PIL import Image
from skimage.io import imread, imsave, imshow, show, imread_collection, imshow_collection
import os, logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'  
from os import listdir
import glob
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
#import keras_tuner as kt
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow import keras
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
os.environ["TF_CPP_VMODULE"]="gpu_process_state=10,gpu_cudamallocasync_allocator=10"
a = tf.zeros([], tf.float32)
## Imports libs

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from IPython.display import clear_output

In [ ]:
def renameImageFiles(folderpath, prefix,fileExtension):
    folder_path = folderpath
    new_prefix = prefix

    for i, file_path in enumerate(glob.glob(folder_path + '*.'+fileExtension)):
        new_file_name = new_prefix + '_' + str(i+1) + '.'+fileExtension
        os.rename(file_path, os.path.join(folder_path, new_file_name))


In [ ]:
path1 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/CT_COVID/'
prefix1 = 'ct_covid'
path2 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/CT_NonCOVID/'
prefix2 = 'ct_noncovid'

# renameImageFiles(path1, prefix1,'png')
# renameImageFiles(path2, prefix2,'png')
# renameImageFiles(path2, prefix2,'jpg')

In [ ]:
def processImages(imgDirPath, binary_label):
    img_names = list()
    try:
        with os.scandir(imgDirPath) as dirs:
            for entry in dirs:
                img_names.append(entry.name)
    except Exception as e:
        print(f"Error while scanning directory: {e}")
        return None

    all_features = []
    for img in img_names:
        try:
            path = imgDirPath + img
            cv_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if cv_img is None:
                print(f"Error reading image: {path}")
                continue

            cv_img2 = cv2.resize(cv_img, (300, 300), interpolation=cv2.INTER_NEAREST)

            nFeatures = (cv_img2.shape[0] * cv_img2.shape[1])
            features = np.reshape(cv_img2, nFeatures)
            all_features.append(features)
        except Exception as e:
            print(f"Error processing image {img}: {e}")

    if len(all_features) == 0:
        print("No valid images found.")
        return None

    imgs_df = pd.DataFrame(np.array(all_features), index=img_names)
    if binary_label == 0:
        imgs_df['class_label'] = np.zeros((imgs_df.shape[0]), dtype=int)
    else:
        imgs_df['class_label'] = np.ones((imgs_df.shape[0]), dtype=int)

    return imgs_df

In [ ]:
ct_covid_features_df =  processImages(path1,1)#1--> covid-19 positive
ct_noncovid_features_df =  processImages(path2,0)#0 ---> covnid-19 negative

In [ ]:
cvd_imgs = [ct_noncovid_features_df, ct_covid_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset)

In [ ]:
ct_noncovid_features_df.shape

In [ ]:
ct_covid_features_df.shape

#### Prepare negative covid images for machine learning ready

In [ ]:
input_data_x_ = ct_noncovid_features_df.iloc[:,:-1].to_numpy().reshape((2173,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_noncovid_features_df['output_encode'] = label_encoder.fit_transform(ct_noncovid_features_df['class_label'])
ct_noncovid_features_df
ct_noncovid_features_df = pd.get_dummies(ct_noncovid_features_df, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y_ = np.array(ct_noncovid_features_df[['class_label']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
input_data_x_negative_only = input_data_x_
output_label_y_negative_only = output_label_y_
print('Input_x Data Shape: \n{0}'.format(input_data_x_negative_only.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_negative_only.shape))

#### Prepare positive covid images for machine learning ready

In [ ]:
input_data_x__ = ct_covid_features_df.iloc[:,:-1].to_numpy().reshape((2476,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_covid_features_df['output_encode'] = label_encoder.fit_transform(ct_covid_features_df['class_label'])
ct_covid_features_df
ct_covid_features_df = pd.get_dummies(ct_covid_features_df, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y__ = np.array(ct_covid_features_df[['class_label']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
input_data_x_positive_only = input_data_x__
output_label_y_positive_only = output_label_y__
print('Input_x Data Shape: \n{0}'.format(input_data_x_positive_only.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_positive_only.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_features_negative_only, test_features_negative_only, train_labels_negative_only, test_labels_negative_only = train_test_split(
    input_data_x_negative_only, output_label_y_negative_only, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_positive_only, test_features_positive_only, train_labels_positive_only, test_labels__positive_only = train_test_split(
    input_data_x_positive_only, output_label_y_positive_only, test_size=.20, random_state=42)

### Loading the dataset

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from torchvision import transforms

class CustomImageDataset(Dataset):
    def __init__(self, features, labels, transform=None):
        """
        Args:
            features (array-like): Input image features (e.g., images as numpy arrays).
            labels (array-like): Corresponding labels for the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        image = self.features[idx]
        label = self.labels[idx]

        # Ensure the image has the correct shape
        if image.ndim == 2:  # Grayscale image (height, width)
            image = np.expand_dims(image, axis=-1)  # Convert to (height, width, 1)
        elif image.ndim == 3 and image.shape[2] == 1:  # If the image is (height, width, 1)
            image = np.squeeze(image, axis=-1)  # Convert to (height, width)

        # Convert to PIL image
        if isinstance(image, np.ndarray):
            if image.ndim == 2:  # Grayscale
                image = Image.fromarray(image)  # Convert from numpy array to PIL Image
            elif image.ndim == 3:  # RGB
                image = Image.fromarray(image.astype('uint8'), 'RGB')

        # Apply transformations if any
        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
from torchvision import transforms

# Custom normalization for grayscale images (mean and std for single channel)
# normalize_grayscale = transforms.Normalize(mean=[0.485], std=[0.229])

# transform = transforms.Compose([
#     transforms.ToTensor(),
#     normalize_grayscale,  # Apply normalization only for grayscale images
# ])

# Use a conditional normalization based on the number of channels
def dynamic_normalize(tensor):
    """
    Normalize tensors dynamically based on the number of channels.
    Args:
        tensor: A PyTorch tensor representing the image.
    Returns:
        Normalized tensor.
    """
    if tensor.shape[0] == 3:  # RGB Image
        return transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(tensor)
    elif tensor.shape[0] == 1:  # Grayscale Image
        return transforms.Normalize(mean=[0.485], std=[0.229])(tensor)
    else:
        raise ValueError(f"Unsupported number of channels: {tensor.shape[0]}")

# Add this dynamic normalization into your transform chain:
transform = transforms.Compose([
    transforms.ToTensor(),
    dynamic_normalize,  # Apply normalization dynamically based on channels
])

### Load datset for NonCOVID

In [ ]:
train_dataset_negative = CustomImageDataset(features=train_features_negative_only, labels=train_labels_negative_only, transform=transform)
#test_dataset = CustomImageDataset(features=test_features, labels=test_labels, transform=transform)

# Create DataLoader instances
loader_negative = DataLoader(train_dataset_negative, batch_size=64, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
type(loader_negative)

In [ ]:
for batch in loader_negative:
    images, labels = batch 
    
    # Convert images from PyTorch tensor to NumPy array
    images_np = images.numpy()
    
    # Convert the NumPy array to TensorFlow tensor
    images_tf = tf.convert_to_tensor(images_np, dtype=tf.float32)
    
print(images_np.shape)

In [ ]:
negative_images = images_tf

In [ ]:
negative_images.shape

### Load datset for COVID

In [ ]:
train_dataset_positive = CustomImageDataset(features=train_features_positive_only, labels=train_labels_positive_only, transform=transform)
#test_dataset = CustomImageDataset(features=test_features, labels=test_labels, transform=transform)

# Create DataLoader instances
loader_positive = DataLoader(train_dataset_positive, batch_size=64, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
for batch in loader_positive:
    images_, labels_ = batch 
    
    # Convert images from PyTorch tensor to NumPy array
    images_np_ = images.numpy()
    #print(images_np_.shape)
    # Convert the NumPy array to TensorFlow tensor
    images_tf_ = tf.convert_to_tensor(images_np, dtype=tf.float32)
    


In [ ]:
positive_images = images_tf_

In [ ]:
type(positive_images)

#### GAN-VAE 

In [ ]:
class UNet(nn.Module):
    def __init__(self, input_channels=1, hidden_dim=64):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, hidden_dim, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_dim, hidden_dim * 2, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_dim * 2, hidden_dim * 4, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(hidden_dim * 4, hidden_dim * 2, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(hidden_dim * 2, hidden_dim, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_dim, input_channels, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

class DiffusionModel:
    def __init__(self, model, timesteps=1000, beta_start=1e-4, beta_end=0.02):
        self.model = model
        self.timesteps = timesteps
        self.betas = torch.linspace(beta_start, beta_end, timesteps)
        self.alphas = 1.0 - self.betas
        self.alpha_bars = torch.cumprod(self.alphas, dim=0)
    
    def forward_diffusion(self, x0, t):
        noise = torch.randn_like(x0)
        sqrt_alpha_bar = torch.sqrt(self.alpha_bars[t])[:, None, None, None]
        sqrt_one_minus_alpha_bar = torch.sqrt(1 - self.alpha_bars[t])[:, None, None, None]
        return sqrt_alpha_bar * x0 + sqrt_one_minus_alpha_bar * noise, noise

    def sample(self, shape, device):
        x = torch.randn(shape, device=device)
        for t in reversed(range(self.timesteps)):
            t_tensor = torch.tensor([t], device=device)
            predicted_noise = self.model(x)
            alpha_bar = self.alpha_bars[t]
            alpha = self.alphas[t]
            beta = self.betas[t]
            x = (1 / torch.sqrt(alpha)) * (x - (beta / torch.sqrt(1 - alpha_bar)) * predicted_noise)
            if t > 0:
                x += torch.sqrt(beta) * torch.randn_like(x)
        return x

In [ ]:
class UNet(nn.Module):
    def __init__(self, input_channels=1, hidden_dim=64):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, hidden_dim, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_dim, hidden_dim * 2, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_dim * 2, hidden_dim * 4, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(hidden_dim * 4, hidden_dim * 2, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(hidden_dim * 2, hidden_dim, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_dim, input_channels, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

class DiffusionModel:
    def __init__(self, model, timesteps=1000, beta_start=1e-4, beta_end=0.02):
        self.model = model
        self.timesteps = timesteps
        self.betas = torch.linspace(beta_start, beta_end, timesteps)
        self.alphas = 1.0 - self.betas
        self.alpha_bars = torch.cumprod(self.alphas, dim=0)
    
    def forward_diffusion(self, x0, t):
        noise = torch.randn_like(x0)
        sqrt_alpha_bar = torch.sqrt(self.alpha_bars[t])[:, None, None, None]
        sqrt_one_minus_alpha_bar = torch.sqrt(1 - self.alpha_bars[t])[:, None, None, None]
        return sqrt_alpha_bar * x0 + sqrt_one_minus_alpha_bar * noise, noise

    def sample(self, shape, device):
        x = torch.randn(shape, device=device)
        for t in reversed(range(self.timesteps)):
            t_tensor = torch.tensor([t], device=device)
            predicted_noise = self.model(x)
            alpha_bar = self.alpha_bars[t]
            alpha = self.alphas[t]
            beta = self.betas[t]
            x = (1 / torch.sqrt(alpha)) * (x - (beta / torch.sqrt(1 - alpha_bar)) * predicted_noise)
            if t > 0:
                x += torch.sqrt(beta) * torch.randn_like(x)
        return x

In [ ]:
# Train the model
train(negative_images, epochs=100)

In [ ]:
print(negative_images.shape)
image_____ = tf.transpose(negative_images, perm=[0, 2, 3, 1])
image.shape

In [ ]:
import numpy as np
from skimage.io import imshow, show

# Ensure the image is a NumPy array with a proper dtype (e.g., uint8 for image data)
for batch in negative_images:
    for i in range(1):
        imgd = batch[i].numpy()  # Convert tensor to NumPy array
        img = np.clip(imgd, 0, 255).astype(np.uint8)  # Ensure the values are within the expected range
        imshow(img)
        show()
